In [2]:
import pandas as pd
import numpy as np
import cPickle 
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import time
import logging
from nltk.stem.snowball import *
import itertools

# Python 2.x:
import HTMLParser
html_parser = HTMLParser.HTMLParser()

import multiprocessing
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import cPickle as pickle
musthave_dice_naruki_data = pickle.load( open( "musthave_dice_naruki_data.p", "rb" ) )
print len(musthave_dice_naruki_data)
musthave_dice_naruki_data = [i for i in musthave_dice_naruki_data if len(i) > 0]
print len(musthave_dice_naruki_data)

110660
110637


In [21]:
import math
idf_obj = {}
idf_keys = []

def tf(word, blob):
    return (1.0 * blob.count(word)) / len(blob)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    if not word in idf_obj:
        idf_obj[word] = math.log(len(bloblist) / (1 + n_containing(word, bloblist)))
    return idf_obj[word]
#         return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [25]:
start = time.time()

musthave_dice_naruki = musthave_dice_naruki_data
all_skills = list(itertools.chain(*musthave_dice_naruki))
all_skills = list(set(all_skills))

num_docs = len(musthave_dice_naruki)
num_skills = len(all_skills)

print 'num_docs: %s\nnum_skills: %s' % (num_docs, num_skills)

td_matrix = pd.DataFrame()
td_matrix['skill'] = all_skills
td_matrix.drop_duplicates(inplace=True)

print "Take: %s (s)" % (time.time() - start)

td_matrix.head()

num_docs: 110637
num_skills: 80695
Take: 0.286262989044 (s)


,skill
0,
1,technical_solution_consultant
2,gse_design
3,semi_solid_topical_fandd
4,clinical_laboratory


In [ ]:
start = time.time()
for ix, tdrow in td_matrix.iterrows():
    for i in range(num_docs):
        doc = 'd' + str(i)
        td_matrix.loc[ix, doc] = tfidf(tdrow['skill'], musthave_dice_naruki[i], musthave_dice_naruki)
print "Take: %s (s)" % (time.time() - start)

td_matrix.to_csv('tfidf_skills_matrix.csv', index=False, encoding='utf-8')